In [ ]:
import psycopg2
import pandas as pd
import warnings
from itertools import combinations
from psycopg2.extras import DictCursor
import numpy as np
warnings.filterwarnings('ignore')

# establish a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="CS712_assignment",
    user="postgres",
    password="3007"
)

# create a cursor object
cur = conn.cursor(cursor_factory=DictCursor)
data = pd.read_csv('dataset2.csv')
table_array=[]
for index, row in data.iterrows():
    id = row['id']
    long = float(row['long'])
    lat = float(row['lat'])
    table_name = row['id'][0]  # Get the first alphabet of the ID
    cur.execute(f"CREATE TABLE IF NOT EXISTS main_table (id VARCHAR(10) , geog GEOGRAPHY(Point, 4326))")
    if table_name not in table_array: # Storing all table names in an array
        table_array.append(table_name)
        cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id VARCHAR(10) , geog GEOGRAPHY(Point, 4326))")
    
    insert = f'''
    INSERT INTO main_table (id, geog) VALUES ('{id}', ST_GeomFromText('POINT({long} {lat})', 4326))
    '''
    cur.execute(insert)  

cur.execute(f"CREATE TABLE IF NOT EXISTS Results ( Co_ordinates VARCHAR(10), Patterns VARCHAR(10))")

# print(table_array)
# conn.commit()



In [ ]:
k=5
trans_step = 0.02
size_step = 0.02
min_side_threshold = 0.02
radius = 500
min_area_threshold = 0.0004 # in degrees
max_area_threshold = 0.002 # in degrees
interest_measure_threshold = 0.15

# Define a function to execute SQL queries and return the results as a pandas dataframe
def execute_query(query):
    df = pd.read_sql_query(query, conn)
    # display(df)
    return df


# Execute the query to retrieve the bounding box of main_table
cur.execute("SELECT ST_XMin(bbox) as xmin, ST_XMax(bbox) as xmax, ST_YMin(bbox) as ymin, ST_YMax(bbox) as ymax FROM (SELECT ST_Extent(ST_Envelope(geog::GEOMETRY )) as bbox FROM main_table) as subquery")

# Fetch the result as a dictionary
result = cur.fetchone()

# Retrieve the four variables xmin, xmax, ymin, ymax from the dictionary
xmin = result['xmin']
xmax = result['xmax']
ymin = result['ymin']
ymax = result['ymax']
print("main table coordinates :  ",xmin,xmax,ymin,ymax)

a=min_side_threshold
b=min_side_threshold
c=xmin
d=ymin
# print(c,c+a,d,d+b)

while a<xmax-xmin:
    b=min_side_threshold
    while b<ymax-ymin:
        c=xmin
        # Area threshold calculation
        area = a*b
        if area < min_area_threshold or area > max_area_threshold :
            break
        while c<xmax:
            d=ymin
            while d<ymax:
                num1={}
                pind={}
                e=c+a
                f=d+b
                if e > xmax :
                    e=xmax
                if f > ymax :
                    f=ymax

                # Making bounding boxex inside the xmin,xmax,ymin,ymax boumding box
                query = (f"""
                        SELECT *
                        FROM main_table
                        WHERE geog && ST_MakeEnvelope({c},{d},{e},{f},4326)
                        """)
                df = execute_query(query)
                
                # inserting data points present inside the newly formed bounding box
                for index, row in df.iterrows():
                    insert = f"""
                    INSERT INTO {row['id'][0]}
                    VALUES ('{row['id']}', '{row['geog']}' );
                    """
                    cur.execute(insert)
                    num1[row['id'][0]]=num1.get(row['id'][0],0)+1



                # All combinations of size 2 tables
                combinations2 = combinations(table_array, 2)
                comb2 = ["".join(i) for i in combinations2]

                # Generate size-2 co-locations 
                for i in range(len(comb2)):
                    # print("Row instance : ", comb2[i])
                    cur.execute(
                        f"CREATE TABLE IF NOT EXISTS {comb2[i]} (id1 VARCHAR(10), id2 VARCHAR(10) )"
                    )
                    query = f"""
                        SELECT a1.id AS id1, b1.id AS id2
                        FROM {comb2[i][0]} AS a1
                        JOIN {comb2[i][1]} AS b1 ON  ST_DWithin(a1.geog, b1.geog, {radius})
                        """
                    df = execute_query(query)
                    for index, row in df.iterrows():
                        insert = f"""
                            INSERT INTO {comb2[i]}
                            VALUES ('{row['id1']}','{row['id2']}');
                            """
                        cur.execute(insert)
                    
                    # calculating Cross-K function and pruning based on interst measure
                    numerator=df.shape[0]
                    if numerator>0:
                        pind[comb2[i]] = numerator/np.prod([num1[x] for x in comb2[i]]) 
                        if pind[comb2[i]] < interest_measure_threshold:
                            query = f"""
                                DELETE FROM {comb2[i]}
                            """
                            cur.execute(query)

                # Generate co-location patterns of size 3 and above 
                for i in range(3, k + 1):
                    combs = ["".join(j) for j in combinations(table_array, i)]
                    for comb in combs:
                        # print("Row instance : ", comb)
                        cur.execute(
                            f"CREATE TABLE IF NOT EXISTS {comb} ({', '.join([f'id{j+1} VARCHAR(10)' for j in range(i)])})"
                        )
                        query = f"""
                            SELECT {', '.join([f'a1.id{j+1} AS id{j+1}' for j in range(i-1)])}, b1.id{i-1} AS id{i}
                            FROM {comb[:-1]} AS a1
                            JOIN {comb[1:]} AS b1 ON {' AND '.join([f'a1.id{j+2}=b1.id{j+1}' for j in range(i-2)])}
                            WHERE EXISTS (
                                SELECT * 
                                FROM {comb[0]}{comb[-1]} AS c1
                                WHERE c1.id1=a1.id1 AND c1.id2=b1.id{i-1}
                                )
                            """
                        df = execute_query(query)

                        for index, row in df.iterrows():
                            insert = f"""
                                INSERT INTO {comb}
                                VALUES ({', '.join([f"'{row[f'id{j+1}']}'" for j in range(i)])});
                                """
                            cur.execute(insert)

                        # calculating Cross-K function and pruning based on interst measure
                        numerator=df.shape[0]
                        if numerator>0:
                            pind[comb] = numerator/np.prod([num1[x] for x in comb]) 
                            if pind[comb] < interest_measure_threshold:
                                query = f"""
                                    DELETE FROM {comb}
                                """
                                cur.execute(query)

                # Pruning of subset patterns if exists
                for i in range(3, k + 1):
                    combs = ["".join(j) for j in combinations(table_array, i)]
                    for comb in combs:
                        prev_combs = ["".join(j) for j in combinations(list(comb), i-1)]
                        for prev_comb in prev_combs:
                            cur.execute(f"""
                                DELETE FROM {prev_comb} WHERE EXISTS (
                                    SELECT * 
                                    FROM {comb} 
                                    )
                                """)
                            
                # Printing the co-ordinates of bounding box if any pattern exists
                for i in range(2, k + 1):
                    combs = ["".join(j) for j in combinations(table_array, i)]
                    for comb in combs:
                        query = f"""
                                    SELECT * FROM {comb} 
                                """
                        df = execute_query(query)
                        if df.shape[0] > 0:
                            print(c,d,e,f)
                            break

                # Printing of patterns
                for i in range(2, k + 1):
                    combs = ["".join(j) for j in combinations(table_array, i)]
                    for comb in combs:
                        query = f"""
                                    SELECT * FROM {comb} 
                                """
                        df = execute_query(query)
                        if df.shape[0] > 0:
                            print(comb)

                # Cleaning tables for next iteration
                for i in range (1,k+1) :
                    combs = ["".join(j) for j in combinations(table_array, i)]
                    for j in combs:
                        query = f"""
                                DELETE FROM {j}
                                """
                        cur.execute(query)

                
                d=d+trans_step
            c=c+trans_step
        b=b+size_step
    a=a+size_step

print("done")

# conn.commit()
cur.close()
conn.close()